## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sp
import pywt

## 데이터 크기 정의하기

In [ ]:
NoOfData    = 100  # 정상/고장 데이터 각 100개씩 
NoOfSensor  = 3    # 반경방향 가속도(Radial-ACC), 축방향 가속도(Axial-ACC), 음향방출(Acoustic Emission)
NoOfFeature = 10   # 추출한 특징 개수 : 10개 (Max, Min, Mean, RMS, Variance, Skewness, Kurtosis, Crest factor, Impact factor, Shape factor)

## Time Domain 특징값 추출 (10 features * 3 sensors = 30개씩)

In [ ]:
def rms(x): # RMS 함수 정의
    return np.sqrt(np.mean(x**2))

In [ ]:
# 특징데이터 크기 지정
TimeFeature_Normal   = np.zeros(shape=(NoOfSensor*NoOfFeature , NoOfData))
TimeFeature_Abnormal = np.zeros(shape=(NoOfSensor*NoOfFeature , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './GearData/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './GearData/Abnormal_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = pd.read_csv(temp_path1 , sep=',' , header=None) # 임시 Normal 데이터
    temp_data2 = pd.read_csv(temp_path2 , sep=',' , header=None) # 임시 Abnormal 데이터
    
    # Time Domain 특징값 추출
    for j in range(NoOfSensor):
        
        # Normal Time Domain Feature
        TimeFeature_Normal[10*j+0, i] = np.max(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+1, i] = np.min(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+2, i] = np.mean(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+3, i] = rms(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+4, i] = np.var(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+5, i] = sp.skew(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+6, i] = sp.kurtosis(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+7, i] = np.max(temp_data1.iloc[:,j+1])/rms(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+8, i] = rms(temp_data1.iloc[:,j+1])/np.mean(temp_data1.iloc[:,j+1])
        TimeFeature_Normal[10*j+9, i] = np.max(temp_data1.iloc[:,j+1])/np.mean(temp_data1.iloc[:,j+1])
        
        # Abnormal Time Domain Feature
        TimeFeature_Abnormal[10*j+0, i] = np.max(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+1, i] = np.min(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+2, i] = np.mean(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+3, i] = rms(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+4, i] = np.var(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+5, i] = sp.skew(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+6, i] = sp.kurtosis(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+7, i] = np.max(temp_data2.iloc[:,j+1])/rms(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+8, i] = rms(temp_data2.iloc[:,j+1])/np.mean(temp_data2.iloc[:,j+1])
        TimeFeature_Abnormal[10*j+9, i] = np.max(temp_data2.iloc[:,j+1])/np.mean(temp_data2.iloc[:,j+1])
        
# Normal, Abnormal 데이터 합치기        
TimeFeature = np.array([TimeFeature_Normal, TimeFeature_Abnormal])

In [ ]:
TimeFeature.shape

## Frequncy Domain 특징값 추출 (10 features \* 12 wavelet levels * 3 sensors = 360개씩)

In [ ]:
# Wavelet options

MotherWavelet = pywt.Wavelet('db4')   # Mother wavelet (모함수) 지정
Level   = 12                    # Wavelet 분해 레벨 지정
select  = 12                    # 특징추출 영역 고주파 영역부터 개수 지정 (d1~)

In [ ]:
#Frequency Domain 특징값 추출 (Wavelet Transform 기반)
FreqFeature_Normal   = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))
FreqFeature_Abnormal = np.zeros(shape=(NoOfSensor*NoOfFeature*select , NoOfData))

for i in range(NoOfData):
    
    # 데이터 불러오기
    temp_path1 = './GearData/Normal_%d'%(i+1)   # Normal 데이터 파일 경로
    temp_path2 = './GearData/Abnormal_%d'%(i+1) # Abnormal 데이터 파일 경로
    temp_data1 = np.array(pd.read_csv(temp_path1 , sep=',', header=None).iloc[:,1:]) # 임시 Normal 데이터
    temp_data2 = np.array(pd.read_csv(temp_path2 , sep=',', header=None).iloc[:,1:]) # 임시 Abnormal 데이터
    Coef1      = pywt.wavedec(temp_data1, MotherWavelet, level=Level, axis=0)
    Coef2      = pywt.wavedec(temp_data2, MotherWavelet, level=Level, axis=0)
    
    # Frequency Domain 특징값 추출
    for j in np.arange(NoOfSensor):
        
        for k in np.arange(select):
            coef1 = Coef1[Level-k]
            coef2 = Coef2[Level-k]
         
            # Normal FD Feature
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+0 , (i-1)] = np.max(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+1 , (i-1)] = np.min(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+2 , (i-1)] = np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+3 , (i-1)] = np.var(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+4 , (i-1)] = rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+5 , (i-1)] = sp.skew(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+6 , (i-1)] = sp.kurtosis(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+7 , (i-1)] = np.max(coef1[:,j])/rms(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+8 , (i-1)] = rms(coef1[:,j])/np.mean(coef1[:,j])
            FreqFeature_Normal[NoOfFeature*j*select+k*NoOfFeature+9 , (i-1)] = np.max(coef1[:,j])/np.mean(coef1[:,j])
            
            # Abnormal FD Feature
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+0 , (i-1)] = np.max(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+1 , (i-1)] = np.min(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+2 , (i-1)] = np.mean(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+3 , (i-1)] = np.var(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+4 , (i-1)] = rms(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+5 , (i-1)] = sp.skew(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+6 , (i-1)] = sp.kurtosis(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+7 , (i-1)] = np.max(coef2[:,j])/rms(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+8 , (i-1)] = rms(coef2[:,j])/np.mean(coef2[:,j])
            FreqFeature_Abnormal[NoOfFeature*j*select+k*NoOfFeature+9 , (i-1)] = np.max(coef2[:,j])/np.mean(coef2[:,j])

# Normal, Abnormal 특징값 합치기            
FreqFeature = np.array([FreqFeature_Normal, FreqFeature_Abnormal])

In [ ]:
FreqFeature.shape

## Feature 통합 (Time domain 30개 + Freq. domain 360 = 390개)

In [ ]:
Features = np.concatenate([TimeFeature,FreqFeature] , axis=1)  #정상/고장 각각: 390개 특징 * 100개 데이터
Features.shape

In [ ]:
# 저장 가능한 데이터 형태로 변환
FeatureData = pd.DataFrame(np.concatenate([Features[0],Features[1]],axis=1))
FeatureData

In [ ]:
# 특징값 저장
path = './SavedData/FeatureData'   # path = '파일 경로/저장할 파일 이름'
FeatureData.to_csv(path, sep=',', header=None , index=None)